# NanoGPT


In [1]:
import torch

In [2]:
import torch.nn as nn 

In [27]:
torch.randint

tensor([[ 0.2908, -0.5701, -0.6286]], grad_fn=<EmbeddingBackward0>)

In [2]:
# loading the shakespear text
with open('data/input.txt','r',encoding='utf-8') as f:
    text=f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print("vocab size:",vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


### Tokenization

We could use:

    - OPenAI tiktoken
    - Google SentencePiece
    - Character level tokenizers

In [5]:
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for i,s in enumerate(chars)}

encode= lambda s:[stoi[c] for c in s]
decode=lambda i:''.join([itos[b] for b in i])

print(encode("hello there !"))
print(decode(encode("hello there !")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 1, 2]
hello there !


In [6]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.type())
print(data[:1000])

torch.Size([1115394]) torch.LongTensor
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
 

In [7]:
n=int(len(data)*0.9)
train_data=data[:n]
val_data=data[n:]

In [8]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
block_size=8
batch_size=4
torch.manual_seed(1337)

def get_batch(split):
    data=  train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x,y

xb,yb=get_batch('train')
print(xb)
print(yb)


for b in range(batch_size):
    for t in range(block_size):
        print(f"when input is {xb[b,:t+1]} the target is {yb[b,t]}")
    


tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target is 1
when input is tensor([44, 53, 56,  1]) the target is 58
w

In [10]:
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx, targets=None):

        logits=self.token_embedding_table(idx)
        if targets is None:
            loss=None
        else:
            # logits shape is (B,T,C) and target shape is (B,T)
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx is (B,T)
        for _ in range(max_new_tokens):
            logits,loss=self(idx) #logits is (B,T,C)
            logits=logits[:,-1,:] # (B,C) only last as context for now
            probs=F.softmax(logits,dim=-1) # (B,C)
            idx_next=torch.multinomial(probs,num_samples=1) #(B,1)
            idx=torch.cat((idx,idx_next),dim=1)
            
        return idx
        




In [77]:

m=BiagramLanguageModel(vocab_size)
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))




$3jZ!&roLfY3PWCFJ$EoH?svg!j&v-,:xpUPu szQkfEYOdG:J,y&clZLpGQ3ANFMGlKuDdQQw-atTY ujb?UldPWfSoyubAWhg:


In [12]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [13]:
batch_size=32
for step in range(10000):
    # sample batch
    xb,yb=get_batch('train')
    # get loss
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step % 1000 ==0:
        print("loss: ",loss)
    

loss:  tensor(4.7040, grad_fn=<NllLossBackward0>)
loss:  tensor(3.7031, grad_fn=<NllLossBackward0>)
loss:  tensor(3.1372, grad_fn=<NllLossBackward0>)
loss:  tensor(2.7768, grad_fn=<NllLossBackward0>)
loss:  tensor(2.5845, grad_fn=<NllLossBackward0>)
loss:  tensor(2.5105, grad_fn=<NllLossBackward0>)
loss:  tensor(2.5316, grad_fn=<NllLossBackward0>)
loss:  tensor(2.5048, grad_fn=<NllLossBackward0>)
loss:  tensor(2.4697, grad_fn=<NllLossBackward0>)
loss:  tensor(2.4839, grad_fn=<NllLossBackward0>)


In [78]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))



w
YfYJPRDUHQmAlFO.xWWjZozQO.xD
y:JEFwz,FldFdoJfi:LaSC,SNt, ?&CAMQiQoCYTXoNXpi&d-HosmYmWlxHJQt$MSeqrW,;-yg!KAY YSDSMdd. m
Y.x!tWO eUUoGXXhtvttbzn-ILIUIYuH&etrRb$:IattatqNszI$t 3rJGKzNLamM
vVpGKicHKwDyASYsBh!
cuaJnxgReUJb!ObIjkqGbB?&bzQUuGdKEdSut$TYzWAQD$ETUFqhUJQy&DHVSJq'mj?&f3jLFkeWbBKaI3AS?,.dLYCUlkfY tswhM-b$Srn mlyK'Em&:wfi:ko;-j;-ameAv$m$m!&CtHK'$:bWBS??&sKtqra MRJKvamW;r !WkDwcmKzN-?,WRBTw$XpLWwsQjPSt?fuiOQqhzBJQmeujZerA;R.i
cSSTo-OdSDUnAyYgR:b$E&cch!MeUxMBS,S?ArD
at uatV!&AYFEly:yZyTlvSe&T


### The mathematical trick in self-attention

In [59]:
torch.manual_seed(1337)
B,T,C=4,8,2
x=torch.randn(B,T,C)

# One self-attention head with dim 16
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)

# generate the key,query and value
k=key(x)
q=query(x)
v=value(x)
wei= q@k.transpose(-2,-1) * C**-0.5

tril=torch.tril(torch.ones(T,T))
# wei=torch.zeros(T,T)
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)
out=wei@ v
out.shape

torch.Size([4, 8, 16])

In [42]:
print(x)
print(wei@x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
      